In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../dataset/gym_members_exercise_tracking.csv")
df.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


In [ ]:
def hydratation_performance_analysys(df: pd.DataFrame) -> dict:

  correlation_session_duration = df["Water_Intake (liters)"].corr(df["Session_Duration (hours)"])
  correlation_avg_bpm = df["Water_Intake (liters)"].corr(df["Avg_BPM"])
  correlation_calories_burned = df["Water_Intake (liters)"].corr(df["Calories_Burned"])

  # Crear categorías
  df["Water_Intake_Category"] = pd.cut(df["Water_Intake (liters)"], bins=[0, 1.5, 3, float('inf')], labels=["Baja 0-1.5L", "Moderada 1.5-3L", "Alta 3L+"])

  grouped_stats = df.groupby("Water_Intake_Category").agg({
      "Calories_Burned": ["mean"],
      "Avg_BPM": ["mean"],
      "Session_Duration (hours)": ["mean"]
  }).reset_index()

  # Aplanar
  grouped_stats.columns = ['_'.join(col).strip() for col in grouped_stats.columns.values]

  result = {
      "result_dataframe": grouped_stats.to_dict(orient="records"),
      "dataframe": [
          {
              "labels": ["Ingesta de Agua", "Calorías Quemadas"] ,
              "correlation": float(correlation_calories_burned * 100),
              "x": grouped_stats["Water_Intake_Category_"].to_list(),
              "y": grouped_stats["Calories_Burned_mean"].to_list(), 
          },
          {
              "labels": ["Ingesta de Agua", "BPM Promedio"] ,
              "correlation": float(correlation_avg_bpm * 100),
              "x": grouped_stats["Water_Intake_Category_"].to_list(), 
              "y": grouped_stats["Avg_BPM_mean"].to_list(),  
          },
          {
              "labels": ["Ingesta de Agua", "Duración de la Sesión"] ,
              "correlation": float(correlation_session_duration * 100),
              "x": grouped_stats["Water_Intake_Category_"].to_list(),
              "y": grouped_stats["Session_Duration (hours)_mean"].to_list(), 
          }
      ]
  }

  return result


In [7]:
result = hydratation_performance_analysys(df)
result_df = pd.DataFrame(result["result_dataframe"])

corr_calories_burned =  result["dataframe"][0]["correlation"]
corr_avg_bpm =  result["dataframe"][1]["correlation"]
corr_session_duration =  result["dataframe"][2]["correlation"]


# graficar
fig = px.line(result_df,
               title=f"Niveles de Ingesta y Calorías Quemadas - Correlación: {corr_calories_burned:.2f} %",
               x="Water_Intake_Category_",
               y="Calories_Burned_mean",
               labels={"Water_Intake_Category_":"Ingesta de Agua", "Calories_Burned_mean":"Calorías Quemadas"},
               )
fig.show()
fig = px.line(result_df,
               title=f"Niveles de Ingesta y BPM Promedio - Correlación: {corr_avg_bpm:.2f} %",
               x="Water_Intake_Category_",
               y="Avg_BPM_mean",
               labels={"Water_Intake_Category_":"Ingesta de Agua", "Avg_BPM_mean":"BPM Promedio"},
               )
fig.show()
fig = px.line(result_df,
               title=f"Niveles de Ingesta y Duración de la Sesión - Correlación: {corr_session_duration:.2f} %",
               x="Water_Intake_Category_",
               y="Session_Duration (hours)_mean",
               labels={"Water_Intake_Category_":"Ingesta de Agua", "Session_Duration (hours)_mean":"Duración de la Sesión"},
               )
fig.show()


<ipython-input-5-29569ed823d0>:10: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

